Recopilación de Datos

- Selección de la Área Geográfica:

Decidiremos una región específica para centrar nuestro estudio, basándonos en la disponibilidad y relevancia de los datos. Por ejemplo, podríamos enfocarnos en una región propensa a incendios forestales.

- Obtención de Datos Meteorológicos y Ambientales:

Usaremos NASA Earthdata para acceder a datos satelitales, para incluir temperatura, humedad y otros factores relevantes buscaremos bases de datos adicionales en estaciones metorológicas para obtenerlos, como registros de incendios forestales históricos y datos meteorológicos locales.

In [1]:
#Tratamiento CSV Data Galicia 

Parte 1: Importar Librerías y Cargar Datos

- Aquí importamos las librerías necesarias y cargamos los dos conjuntos de datos. 

- Visualizamos los primeros registros para entender su estructura.

In [2]:
import pandas as pd
from sklearn.impute import KNNImputer
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy import stats

# Rutas a los archivos CSV originales
ruta_galicia_inc = 'Data/data_Galicia/galicia_inc.csv'
ruta_incen_gal_neg1 = 'Data/data_Galicia/incen_gal_neg_est.csv'

# Cargar los datasets
galicia_inc_df = pd.read_csv(ruta_galicia_inc)
incen_gal_neg1_df = pd.read_csv(ruta_incen_gal_neg1)

# Visualizar los primeros registros para comprender la estructura de los datos
print("Primeros registros de galicia_inc_df:")
print(galicia_inc_df.head(), "\n")

print("Primeros registros de incen_gal_neg1_df:")
print(incen_gal_neg1_df.head(), "\n")

# Resumen: En esta parte hemos importado las librerías necesarias y cargado los datos de los dos archivos CSV. 
# Hemos visualizado los primeros registros para obtener una vista preliminar de la estructura de los datos.

Primeros registros de galicia_inc_df:
   latitude  longitude  brightness  scan  track    acq_date  acq_time  \
0   42.3199    -6.9259       302.6   1.1    1.0  2008-01-24      1118   
1   42.3272    -6.9307       303.2   1.1    1.0  2008-01-24      1118   
2   42.3128    -6.9273       302.6   2.1    1.4  2008-01-24      1259   
3   42.1421    -7.1594       312.9   1.1    1.0  2008-01-25      1342   
4   41.9555    -7.9716       303.3   1.0    1.0  2008-01-25      1342   

  satellite instrument  confidence  ... Provincia  Comunidad_Autonoma  \
0     Terra      MODIS          53  ...   Ourense             Galicia   
1     Terra      MODIS          55  ...   Ourense             Galicia   
2      Aqua      MODIS          53  ...   Ourense             Galicia   
3      Aqua      MODIS          73  ...   Ourense             Galicia   
4      Aqua      MODIS          52  ...   Ourense             Galicia   

           Estacion       Fecha  Humedad foliar Humedad relativa  \
0           a ve

Parte 2: Unificación de Datasets con la Columna 'Incendio'

- Vamos a unir ambos datasets. Es importante verificar las columnas para entender cómo se combinan los datos. 

- Incluir la adición de la columna 'incendio' antes de unificar los datasets. Este paso es esencial para mantener un seguimiento adecuado de los datos de incendios forestales.

In [3]:
# Añadir la columna 'incendio' con valores 1 al DataFrame galicia_inc_df
galicia_inc_df['incendio'] = 1

# Añadir la columna 'incendio' con valores 0 al DataFrame incen_gal_neg1_df si no existe
if 'incendio' not in incen_gal_neg1_df.columns:
    incen_gal_neg1_df['incendio'] = 0

# Unir ambos DataFrames
df_unido = pd.concat([galicia_inc_df, incen_gal_neg1_df], ignore_index=True)

# Visualización de las columnas resultantes
print("Columnas después de unificar los datasets:")
print(df_unido.columns.tolist(), "\n")

# Resumen: Hemos añadido la columna 'incendio' a ambos datasets para marcar los registros como pertenecientes a incendios o no. 
# Luego hemos unificado ambos datasets en uno solo para facilitar el análisis y procesamiento subsiguiente.


Columnas después de unificar los datasets:
['latitude', 'longitude', 'brightness', 'scan', 'track', 'acq_date', 'acq_time', 'satellite', 'instrument', 'confidence', 'version', 'bright_t31', 'frp', 'daynight', 'type', 'location_info', 'Provincia', 'Comunidad_Autonoma', 'Estacion', 'Fecha', 'Humedad foliar', 'Humedad relativa', 'Humedad suelo', 'Precipitación', 'Temperatura del suelo', 'Viento', 'incendio', 'municipio'] 



Parte 3: Limpieza de Datos

- Aquí eliminaremos las columnas innecesarias y trataremos los valores nulos. También agregaremos una barra de progreso para visualizar este proceso.

In [4]:
# Columnas a eliminar
columnas_a_eliminar = ['version', 'type', 'instrument', 'satellite']

# Eliminar las columnas del DataFrame unificado
for col in tqdm(columnas_a_eliminar, desc="Eliminando columnas innecesarias"):
    if col in df_unido.columns:
        df_unido.drop(columns=[col], inplace=True)

# Añadir la columna 'incendio' si es necesario
if 'incendio' not in df_unido.columns:
    df_unido['incendio'] = 0  # Asumiendo que 'incendio' debería ser 0 si no existe

# Visualizar el DataFrame después de la limpieza
print("DataFrame después de la limpieza:")
print(df_unido.head(), "\n")

# Resumen: Hemos eliminado columnas que no son necesarias para nuestro análisis y hemos añadido la columna 'incendio' donde era necesario. 
# Esto ayuda a simplificar el dataset y prepararlo para el análisis.

Eliminando columnas innecesarias: 100%|██████████| 4/4 [00:00<00:00, 108.16it/s]

DataFrame después de la limpieza:
   latitude  longitude  brightness  scan  track    acq_date  acq_time  \
0   42.3199    -6.9259       302.6   1.1    1.0  2008-01-24      1118   
1   42.3272    -6.9307       303.2   1.1    1.0  2008-01-24      1118   
2   42.3128    -6.9273       302.6   2.1    1.4  2008-01-24      1259   
3   42.1421    -7.1594       312.9   1.1    1.0  2008-01-25      1342   
4   41.9555    -7.9716       303.3   1.0    1.0  2008-01-25      1342   

   confidence  bright_t31   frp  ...          Estacion       Fecha  \
0          53       282.2   8.7  ...           a veiga  2008-01-24   
1          55       281.7   8.6  ...           a veiga  2008-01-24   
2          53       283.8  21.3  ...           a veiga  2008-01-24   
3          73       273.3  17.1  ...           a veiga  2008-01-25   
4          52       283.6   7.3  ...  calvos de randin  2008-01-25   

  Humedad foliar Humedad relativa Humedad suelo Precipitación  \
0            0.0             98.0       -

In [5]:
df_unido

,latitude,longitude,brightness,scan,track,acq_date,acq_time,confidence,bright_t31,frp,...,Estacion,Fecha,Humedad foliar,Humedad relativa,Humedad suelo,Precipitación,Temperatura del suelo,Viento,incendio,municipio
0,42.319900,-6.925900,302.6,1.1,1.0,2008-01-24,1118,53,282.2,8.7,...,a veiga,2008-01-24,0.0,98.0,-9999.0,0.0,2.8,30.0,1,NaN
1,42.327200,-6.930700,303.2,1.1,1.0,2008-01-24,1118,55,281.7,8.6,...,a veiga,2008-01-24,0.0,98.0,-9999.0,0.0,2.8,30.0,1,NaN
2,42.312800,-6.927300,302.6,2.1,1.4,2008-01-24,1259,53,283.8,21.3,...,a veiga,2008-01-24,0.0,98.0,-9999.0,0.0,2.8,30.0,1,NaN
3,42.142100,-7.159400,312.9,1.1,1.0,2008-01-25,1342,73,273.3,17.1,...,a veiga,2008-01-25,0.0,93.0,-9999.0,0.0,3.7,28.9,1,NaN
4,41.955500,-7.971600,303.3,1.0,1.0,2008-01-25,1342,52,283.6,7.3,...,calvos de randin,2008-01-25,13.0,73.0,38.9,0.0,5.3,5.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16893,41.866827,-7.453686,0.0,0.0,0.0,2012-03-13,528,0,0.0,0.0,...,rios,2012-03-13,0.0,57.0,0.3,3.2,10.2,10.7,0,NaN
16894,42.291368,-8.277369,0.0,0.0,0.0,2018-07-30,1018,0,0.0,0.0,...,castrelo de mino,2018-07-30,0.1,72.0,0.0,0.0,23.1,3.1,0,NaN
16895,43.600801,-7.600368,0.0,0.0,0.0,2016-02-06,247,0,0.0,0.0,...,mondonedo,2016-02-06,10.4,88.0,0.5,27.8,8.7,4.9,0,NaN
16896,43.089310,-8.695600,0.0,0.0,0.0,2017-05-25,1702,0,0.0,0.0,...,val do dubra,2017-05-25,4.1,96.0,0.2,14.4,17.3,7.2,0,NaN


Parte 4: Análisis Exploratorio y Tratamiento de Valores -9999

- En esta sección, realizamos un análisis exploratorio básico para obtener una comprensión inicial de los datos y reemplazamos los valores -9999 con NaN, que es un paso crucial para el correcto manejo de valores faltantes en los análisis posteriores. 

- También visualizamos un resumen estadístico.

In [6]:
# Análisis Exploratorio Básico
print("Resumen estadístico de los datos numéricos:")
print(df_unido.describe(), "\n")

# Reemplazo de valores -9999 con np.nan
for col in df_unido.select_dtypes(include=['float64', 'int64']).columns:
    df_unido[col] = df_unido[col].apply(lambda x: np.nan if x == -9999 else x)

print("Valores -9999 reemplazados por NaN")

# Resumen: Hemos realizado un análisis exploratorio básico para obtener un resumen estadístico de los datos. 
# También hemos reemplazado los valores -9999 por NaN para facilitar el manejo de valores faltantes en pasos posteriores.

Resumen estadístico de los datos numéricos:
           latitude     longitude    brightness          scan         track  \
count  16898.000000  16898.000000  16898.000000  16898.000000  16898.000000   
mean      42.591582     -7.853686    163.377304      0.862992      0.623832   
std        0.476758      0.622673    163.878099      1.062603      0.650101   
min       39.481900     -9.322227      0.000000      0.000000      0.000000   
25%       42.198122     -8.315754      0.000000      0.000000      0.000000   
50%       42.522065     -7.797150    300.100000      1.000000      1.000000   
75%       42.949975     -7.321125    319.000000      1.300000      1.100000   
max       43.838286     -0.574700    505.400000      4.800000      2.000000   

           acq_time    confidence    bright_t31           frp  Humedad foliar  \
count  16898.000000  16898.000000  16898.000000  16898.000000    16898.000000   
mean    1287.501894     37.025920    146.431477     35.521075     -787.905261   
s

Parte 5: Imputación KNN

- Ahora, utilizaremos el KNN Imputer para imputar los valores faltantes en las columnas relacionadas con el clima. 

- La imputación KNN es un método efectivo para tratar los valores faltantes en conjuntos de datos, especialmente cuando estos valores no son aleatorios.

In [7]:
from sklearn.impute import KNNImputer

# Creando el imputador KNN
knn_imputer = KNNImputer(n_neighbors=5)

# Seleccionando solo las columnas relacionadas con el clima para la imputación KNN
columnas_clima = ['Temperatura del suelo', 'Humedad relativa', 'Precipitación', 'Viento', 'Humedad foliar', 'Humedad suelo']
data_clima = df_unido[columnas_clima]

# Aplicando la imputación KNN a las columnas de clima
data_imputed_knn_clima = knn_imputer.fit_transform(data_clima)

# Convirtiendo los datos imputados de clima de nuevo a un DataFrame y actualizando el DataFrame original
data_imputed_knn_clima = pd.DataFrame(data_imputed_knn_clima, columns=columnas_clima)
df_unido[columnas_clima] = data_imputed_knn_clima

print("Imputación KNN completada en las columnas de clima.")

# Resumen: Utilizamos KNN Imputer para tratar los valores faltantes en las columnas de clima. 
# Esto ayuda a mejorar la integridad de nuestro dataset, permitiendo análisis más precisos en etapas posteriores.

Imputación KNN completada en las columnas de clima.


Parte 6: Detección y Manejo de Outliers

- Finalmente, identificaremos y manejaremos los outliers en nuestro dataset. 

- Los outliers pueden afectar significativamente los resultados de muchos modelos de aprendizaje automático y estadísticas, por lo que es importante detectarlos y decidir cómo manejarlos.

In [8]:
# Detección de outliers usando el método de puntuación Z
z_scores = np.abs(stats.zscore(df_unido.select_dtypes(include=['float64', 'int64'])))
outliers = (z_scores > 3).any(axis=1)

# Visualización de outliers
print(f"Total de outliers detectados: {outliers.sum()}")
print("Primeros 5 registros de outliers:")
print(df_unido[outliers].head(), "\n")

# Opciones de manejo: eliminar, transformar o mantener los outliers. 
# Por ejemplo, aquí los eliminamos:
df_sin_outliers = df_unido[~outliers]

print("Dataset después de eliminar outliers:")
print(df_sin_outliers.head(), "\n")

# Resumen: Hemos detectado y manejado los outliers en nuestro dataset. 
# Dependiendo del análisis y de los objetivos, los outliers pueden ser eliminados, transformados o conservados.

Total de outliers detectados: 1798
Primeros 5 registros de outliers:
    latitude  longitude  brightness  scan  track    acq_date  acq_time  \
0    42.3199    -6.9259       302.6   1.1    1.0  2008-01-24      1118   
1    42.3272    -6.9307       303.2   1.1    1.0  2008-01-24      1118   
2    42.3128    -6.9273       302.6   2.1    1.4  2008-01-24      1259   
3    42.1421    -7.1594       312.9   1.1    1.0  2008-01-25      1342   
11   42.3614    -8.3767       301.7   1.0    1.0  2008-01-29      1137   

    confidence  bright_t31   frp  ...    Estacion       Fecha Humedad foliar  \
0           53       282.2   8.7  ...     a veiga  2008-01-24            0.0   
1           55       281.7   8.6  ...     a veiga  2008-01-24            0.0   
2           53       283.8  21.3  ...     a veiga  2008-01-24            0.0   
3           73       273.3  17.1  ...     a veiga  2008-01-25            0.0   
11          48       286.1   6.6  ...  ponteareas  2008-01-29           83.0   

   Hu

Parte 7: Guardar Datos y Verificación Final

- En esta última parte, guardaremos el dataset procesado y realizaremos una verificación final para confirmar que no quedan valores -9999 en el dataset. 

- Esta etapa es crucial para asegurar que los datos estén listos para análisis y modelado posteriores.

In [9]:
# Guardar el DataFrame procesado en un nuevo archivo CSV
file_path_procesado = 'Data/data_Galicia/dataset_unificado_imputed.csv'
df_sin_outliers.to_csv(file_path_procesado, index=False)
print("El dataset procesado ha sido guardado en 'Data/data_Galicia/dataset_unificado_imputed.csv'.")

# Verificar la presencia de valores -9999 en el dataset final
print("Verificando la presencia de valores -9999 en el dataset procesado:")
if (df_sin_outliers == -9999).any().any():
    print("Se encontraron valores -9999 en el dataset.")
else:
    print("No se encontraron valores -9999 en el dataset.")


El dataset procesado ha sido guardado en 'Data/data_Galicia/dataset_unificado_imputed.csv'.
Verificando la presencia de valores -9999 en el dataset procesado:
No se encontraron valores -9999 en el dataset.


In [10]:
df_sin_outliers

,latitude,longitude,brightness,scan,track,acq_date,acq_time,confidence,bright_t31,frp,...,Estacion,Fecha,Humedad foliar,Humedad relativa,Humedad suelo,Precipitación,Temperatura del suelo,Viento,incendio,municipio
4,41.955500,-7.971600,303.3,1.0,1.0,2008-01-25,1342,52,283.6,7.3,...,calvos de randin,2008-01-25,13.0,73.0,38.9,0.0,5.3,5.0,1,NaN
5,42.209200,-7.237900,315.2,1.0,1.0,2008-01-27,1329,66,284.1,15.7,...,a veiga,2008-01-27,0.0,62.0,0.0,0.0,3.3,14.0,1,NaN
6,42.216700,-7.245400,303.1,1.0,1.0,2008-01-27,1329,48,284.5,7.5,...,a veiga,2008-01-27,0.0,62.0,0.0,0.0,3.3,14.0,1,NaN
7,41.971600,-7.034800,305.1,1.0,1.0,2008-01-27,1329,61,287.7,7.6,...,rios,2008-01-27,0.0,78.0,0.3,19.4,5.5,5.3,1,NaN
8,41.973400,-7.022800,302.5,1.0,1.0,2008-01-27,1329,52,287.1,6.0,...,rios,2008-01-27,0.0,78.0,0.3,19.4,5.5,5.3,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16893,41.866827,-7.453686,0.0,0.0,0.0,2012-03-13,528,0,0.0,0.0,...,rios,2012-03-13,0.0,57.0,0.3,3.2,10.2,10.7,0,NaN
16894,42.291368,-8.277369,0.0,0.0,0.0,2018-07-30,1018,0,0.0,0.0,...,castrelo de mino,2018-07-30,0.1,72.0,0.0,0.0,23.1,3.1,0,NaN
16895,43.600801,-7.600368,0.0,0.0,0.0,2016-02-06,247,0,0.0,0.0,...,mondonedo,2016-02-06,10.4,88.0,0.5,27.8,8.7,4.9,0,NaN
16896,43.089310,-8.695600,0.0,0.0,0.0,2017-05-25,1702,0,0.0,0.0,...,val do dubra,2017-05-25,4.1,96.0,0.2,14.4,17.3,7.2,0,NaN
